# Load Packages

In [1]:

%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
import torch
sys.path.append("../../")

from src.file_manager.load_data import load_split_dict
from src.file_processing.processing_predictions import load_prediction_df_dict, save_pi_df_dict
from src.file_manager.save_load_scaler import load_scaler
from src.pi_methods.knn import knn_prediction_interval
from src.pi_methods.weighted import weighted_prediction_interval
from src.pi_methods.cond_gaussian import cond_gauss_prediction_interval
from src.pi_methods.gaussian_copula import gauss_copula_prediction_interval
from src.pi_methods.conformal_prediction import conformal_prediction_interval
from src.misc import create_folder
from ue_pi_dicts import ue_dict
from seed_file import seed

# seed = 2023
data_label = "physionet"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_processed_data_folder = join(fp_project_folder, "processed_data")
fp_output_data_folder = join(fp_processed_data_folder, "physionet")
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")
fp_pi_predictions = join(fp_project_checkpoints, "pi_predictions")

# Seed filepaths
fp_cur_model_folder = join(fp_models, str(seed))
create_folder(fp_cur_model_folder)
fp_cur_predictions_folder = join(fp_predictions, str(seed))
create_folder(fp_cur_predictions_folder)
fp_cur_pi_predictions_folder = join(fp_pi_predictions, str(seed))
create_folder(fp_cur_pi_predictions_folder)

pred_file_names =  ["rue", "gpr", "infernoise", "der", "bnn"]

2025-06-05 14:29:06.411436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-05 14:29:06.423320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-05 14:29:06.426903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-05 14:29:06.436832: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-05 14:29:07.059629: W tensorflow/compiler/tf2

# Load Data

In [2]:
split_dict = load_split_dict(fp_output_data_folder)

# Load All Predictions

In [3]:
scaler = load_scaler(join(fp_output_data_folder, "scaler.pickle"))
pred_df_dict = load_prediction_df_dict(
    split_dict, fp_cur_predictions_folder, pred_file_names=pred_file_names) 

  0%|          | 0/3 [00:00<?, ?it/s]

# Add KNN PI

In [13]:
# Apply knn_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = knn_prediction_interval(
        df_val=val_df, df_test=test_df, 
        predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )


  0%|          | 0/3 [00:00<?, ?it/s]

# Add Weighted Percentile PI

In [14]:
# Apply weighted_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = weighted_prediction_interval(
        df_val=val_df, df_test=test_df, 
        predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )

  0%|          | 0/3 [00:00<?, ?it/s]

0
1.5339514699400601e-06


  0%|          | 0/5 [00:00<?, ?it/s]

0
1.303178103646748e-06


  0%|          | 0/5 [00:00<?, ?it/s]

0
1.3402297783367728e-06


  0%|          | 0/5 [00:00<?, ?it/s]

# Add Cond Gauss PI

In [15]:
# Apply cond_gauss_prediction_interval to all RUE
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = cond_gauss_prediction_interval(
        df_val=val_df, df_test=test_df, predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

# Add Gaussian Copula PI

In [16]:
ue_label = "RUE"
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    pred_label, ue_col = ue_dict[ue_label]["pred_label"], ue_dict[ue_label]["ue_col"]
    pred_df_dict[time_label]["test_df"] = gauss_copula_prediction_interval(
        df_val=val_df, df_test=test_df, 
        predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
        pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
        scaler=scaler, seed=seed
    )

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

# Add CP

In [17]:
# Apply conformal prediction to all UEs
for time_label, time_info in tqdm(pred_df_dict.items()):
    val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
    for ue_label, ue_info in ue_dict.items():
        val_df, test_df, pred_cols = time_info["valid_df"], time_info["test_df"], time_info["pred_cols"]
        pred_label, ue_col = ue_info["pred_label"], ue_info["ue_col"]
        pred_df_dict[time_label]["test_df"] = conformal_prediction_interval(
            df_val=val_df, df_test=test_df, 
            predictors=split_dict["feat_cols"], pred_cols=pred_cols, 
            pred_label=pred_label, regressor_label=time_label, ue_col=ue_col, 
            scaler=scaler, seed=seed
        )

  0%|          | 0/3 [00:00<?, ?it/s]

# Export df dict

In [18]:
save_pi_df_dict(pred_df_dict, fp_cur_pi_predictions_folder)

  0%|          | 0/3 [00:00<?, ?it/s]

Saved df_dict!
